# Imports, define solve function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

In [ ]:
def solve_AllenCahn(u0=None, L=1.0, N=200, dt=0.1, steps=500, epsilon=0.1, mu=1.0, rng=None, boundary_condition='Neumann'):
    if u0 is None:
        if rng is None:
            rng = np.random.default_rng()
        u = rng.uniform(-1, 1, N) / 1000  # Random initial condition
    else:
        u = u0

    dx = L / N        # Spatial step size
    x = np.linspace(0, L, N, endpoint=False)

    # Construct Laplacian with periodic boundary conditions
    main_diag = -2.0 * np.ones(N)
    off_diag = np.ones(N - 1)
    laplacian = diags([off_diag, main_diag, off_diag], offsets=[-1, 0, 1], shape=(N, N)).toarray()

    if boundary_condition in ['Neumann', 'neumann']:
        laplacian[0, 1] = 2.0   # Neumann BC at left boundary
        laplacian[-1, -2] = 2.0 # Neumann BC at right boundary
    elif boundary_condition in ['periodic', 'Periodic']:
        laplacian[0, -1] = laplacian[-1, 0] = 1.0
    else:
        raise NotImplementedError("Unknown boundary condition", boundary_condition)

    laplacian = laplacian / dx**2

    # Convert to sparse matrix for efficiency
    L_eps = epsilon**2 * laplacian
    I = np.eye(N)
    A = I - dt * L_eps  # Implicit matrix for diffusion

    # Time integration loop
    snapshots = np.empty((steps+1, N))
    snapshots[0] = u
    for n in range(steps):
        f_u = (u**3 - mu*u)  # Nonlinear term
        rhs = u - dt * f_u
        u = spsolve(A, rhs)
        # if n % 100 == 0:
        if True:
            snapshots[n+1] = u

    return snapshots

# Create pictures

In [ ]:
rng = np.random.default_rng(42)  # For reproducibility

N = 200
steps = 1000
dt = 0.1

for i in range(10):

    # Solve the Allen-Cahn equation
    sol = solve_AllenCahn(rng=rng, epsilon=1e-2, mu=1.0, steps=steps, dt=dt, N=N, boundary_condition='periodic')

    plt.figure(dpi=200)
    plt.imshow(sol.T, vmin=-1, vmax=1, aspect='auto', cmap='coolwarm')
    plt.xlabel('Time')
    plt.ylabel('Space')
    plt.savefig(f'AllenCahn_illustrative_figures_{i}.png')
    plt.colorbar(label='u(x,t)')
    plt.show()

# Create dataset

In [ ]:
rng = np.random.default_rng(42)  # For reproducibility

# Parameters
N_samples = 2000
epsilon_arr = 10**rng.uniform(-3, -1, N_samples)

# Bifurcation parameter mu: 1/50 of samples in [-0.1, 0], rest in [0, 1]
# because negative values of mu are less interesting
mu_arr1 = rng.uniform(-0.1, 0.0, N_samples//50)
mu_arr2 = rng.uniform(0.0, 1.0, N_samples - N_samples//50)
mu_arr = np.concatenate([mu_arr1, mu_arr2])

N = 200
steps = 1000
dt = 0.1
solutions = np.empty((len(epsilon_arr), steps+1, N))

for i, [epsilon, mu] in enumerate(zip(epsilon_arr, mu_arr)):
    print(f'{i:<4} Solving for epsilon = {epsilon}, mu = {mu}')

    # Solve the Allen-Cahn equation
    sol = solve_AllenCahn(rng=rng, epsilon=epsilon, mu=mu, steps=steps, dt=dt, N=N, boundary_condition='periodic')
    solutions[i] = sol

In [ ]:
print('solution shape:', solutions.shape)
print('mu_arr.shape:', mu_arr.shape)
print('epsilon_arr.shape:', epsilon_arr.shape)

In [ ]:
# Convert to torch tensors and save

import torch
import pickle

# Convert solutions to torch tensors
solutions2 = torch.tensor(solutions, dtype=torch.float32)
epsilon_arr2 = torch.tensor(epsilon_arr, dtype=torch.float32)
mu_arr2 = torch.tensor(mu_arr, dtype=torch.float32)

# Save
with open(f'../data/AllenCahn_data_periodic_{N_samples}_2.pkl', 'wb') as f:
    pickle.dump({
        'solutions': solutions2,
        'epsilon': epsilon_arr2,
        'mu': mu_arr2
    }, f)

# Count nr of blobs

In [ ]:
import pickle
import torch
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def count_sign_switches(arr, cutoff=0.1):
    """Count the number of sign switches in a tensor along the last dimension, ignoring small values (negative or positive) close to zero. Assumes arr is periodic in the last dimension.

    Parameters
    ----------
    arr : torch.Tensor, shape [..., N]
        Input tensor.
    cutoff : float, optional
        Threshold for considering a value as zero, by default 0.1

    Returns
    -------
    torch.Tensor, shape [..., ]
        A tensor containing the count of sign switches for each row. Same shape as `arr` but with the last dimension removed.
    """
    shape = arr.shape  # [..., N]
    arr = arr.view(-1, shape[-1])  # shape [M, N]

    # Apply cutoff to treat small values as zero
    arr_temp = torch.sign(arr)  # still shape [M, N]
    arr_temp[torch.abs(arr) < cutoff] = 0.0

    # Replace zeros with NaN to ignore them
    arr_no_zeros = arr_temp.clone()  # also shape [M, N]
    arr_no_zeros[arr_no_zeros == 0] = float('nan')

    # Forward fill NaNs along each row
    mask = ~torch.isnan(arr_no_zeros)  # booleans, also shape [M, N]
    idx = torch.arange(arr_temp.size(1), device=arr.device).repeat(arr_temp.size(0), 1)  # shape [M, N], e.g. [[0, 1, 2], [0, 1, 2]]
    idx[~mask] = 0  # still shape [M, N], e.g. [[0, 0, 2], [0, 1, 0]]
    idx = torch.cummax(idx, dim=1)[0]   # shape [M, N], e.g. [[0, 0, 2], [0, 1, 1]]
    filled = torch.gather(arr_no_zeros, dim=1, index=idx)  # shape [M, N]
    # filled now has NaNs replaced by the last non-NaN value before it in the row. Exception is if the first element is a NaN value.
    # print('forward filled:\n', filled)

    # Backward fill NaNs along each row
    filled = torch.flip(filled, dims=[1])
    mask = ~torch.isnan(filled)  # booleans, also shape [M, N]
    idx = torch.arange(arr_temp.size(1), device=arr.device).repeat(arr_temp.size(0), 1)
    idx[~mask] = 0
    idx = torch.cummax(idx, dim=1)[0]
    # print(idx.shape)
    filled = torch.gather(filled, dim=1, index=idx)  # shape [M, N]
    # print('backward filled:\n', filled)

    # Compare signs of adjacent elements
    sign_changes = (torch.sign(filled[:, 1:]) != torch.sign(filled[:, :-1])) & \
                   ~torch.isnan(filled[:, 1:]) & ~torch.isnan(filled[:, :-1])
    sign_changes = sign_changes.sum(dim=1)

    # print('sign changes:\n', sign_changes)
    # print(sign_changes.shape)
    # print('last element:\n')
    # print(torch.sign(filled[:, 0]))
    # print(torch.sign(filled[:, 1]))
    # print(torch.sign(filled[:, 0]) != torch.sign(filled[:, -1]))
    sign_changes += torch.sign(filled[:, 0]) != torch.sign(filled[:, -1])

    # print('sign_changes incl. last element:\n', sign_changes)

    # Count sign changes per row
    return sign_changes.view(shape[:-1])

# test: should give tensor([0, 2, 0])
arr = torch.tensor([[0.05, -0.05, 0.09, -0.08, 0.02, -0.01],
                    [0.05, -0.05, 0.9, -0.08, 0.02, -0.16],
                    [0.05, -0.05, 0.09, -0.08, 0.02, -0.01]])
count_sign_switches(arr)

In [ ]:
# Open
with open(f'../data/AllenCahn_data_periodic_2000_2.pkl', 'rb') as f:
    data = pickle.load(f)
    solutions = data['solutions']
    epsilon_arr = data['epsilon']
    mu_arr = data['mu']

In [ ]:
with torch.no_grad():
    n_sign_switches = count_sign_switches(solutions[:, -1], cutoff=1e-4)  # use only last time step

In [ ]:
counts = np.bincount(n_sign_switches.cpu().numpy())
plt.bar(range(len(counts)), counts)
plt.xlabel('Number of Sign Switches')
plt.ylabel('Frequency')
plt.title('Distribution of Sign Switches in Solutions')
plt.show()

In [ ]:
n_sign_switches.shape

In [ ]:
epsilon_arr.shape

In [ ]:
plt.figure(figsize=(3,3), dpi=200)
plt.scatter(epsilon_arr, n_sign_switches, s=3, alpha=0.1)
plt.xlabel('$\epsilon$')
plt.ylabel('Number of sign switches\nin last time step')